  <h1 style="color: brown;">1. Introduction</h1>

<div style="background-color: #F5F5DC; border: 2px solid #8b4513; padding: 15px; border-radius: 10px; font-family: Arial, sans-serif; box-shadow: 3px 3px 15px rgba(0, 0, 0, 0.1);">

  <p>The objective of this task is to solve the Travelling Salesman Problem (TSP) consisting of 200 cities using the Ant Colony Optimization (ACO) algorithm. The TSP involves determining the shortest possible route that visits each city exactly once and returns to the starting city. ACO, inspired by the foraging behavior of ants, uses a pheromone-based probabilistic approach to find optimal or near-optimal solutions for such combinatorial problems.</p>

  <p>This task involves answering the following key questions:</p>

  <ul>
    <li><strong>Effects of Heuristic Strength</strong>: The first objective is to analyze how the heuristic strength parameters influence the quality of the solutions.</li>
    <li><strong>Effects of Pheromone Evaporation Rate</strong>: The second objective is to investigate how varying the pheromone evaporation rate affects the quality of the solutions.</li>
    <li><strong>Comparison with GA</strong>: The final objective is to compare the best solution obtained using ACO with the best solution obtained using Genetic Algorithm in the first assignment.</li>
  </ul>

</div>

<h1 style="color:brown;">2. Importing Required Libraries</h1>

In [5]:
# Importing Libraries
import numpy as np
import matplotlib.pyplot as plt
from readTSP import *

<h1 style="color:brown;">3. Function to Initialize Ants' Starting Positions</h1>

In [7]:
def initializeAnts(cities, colony):
    # Generating an array of random integers to represent the starting city for each ant
    return np.random.randint(cities.shape[0], size=colony)

<h1 style="color:brown;">4. Function to Calculate Inverse Distances Matrix</h1>

In [8]:
def inverseDistances(cities):
    # Creating a 2-dimensional array filled with zeros, where each element will store the distance between cities
    distances = np.zeros((cities.shape[0], cities.shape[0]))

    # Iterating over each city (point) to calculate the distance to every other city
    for index, point in enumerate(cities):
        # Calculating the Euclidean distance between the current city ('point') and all other cities
        # Using broadcasting to subtract the current city coordinates from all city coordinates, then squaring, summing, and square-rooting
        distances[index] = np.sqrt(((cities - point) ** 2).sum(axis=1))

    # Suppressing warnings about invalid floating-point operations (like division by zero or taking inverse of zero)
    with np.errstate(all='ignore'):
        # Inverting the distances to get the inverse distance matrix
        inv_distances = 1 / distances
    
    # Replacing any infinite values in the inverse distance matrix with zero to avoid issues with zero division
    inv_distances[inv_distances == np.inf] = 0

    # Returning the matrix of inverse distances
    return inv_distances

<h1 style="color:brown;">5. Function to Move Ants and Generate Paths</h1>

In [9]:
def moveAnts(cities, positions, inv_distances, pheromones, alpha, beta, del_tau):
    # Initializing an array to store the paths of all ants
    paths = np.zeros((cities.shape[0], positions.shape[0]), dtype=int) - 1

    # Setting the starting positions of all ants in the paths array
    paths[0] = positions

    # Looping through each node (city) from the second node to the last
    for node in range(1, cities.shape[0]):
        # Looping through each ant
        for ant in range(positions.shape[0]):
            # Calculating the probability of traveling to each next node based on pheromone levels and inverse distances
            next_location_probability = (inv_distances[positions[ant]] ** alpha + pheromones[positions[ant]] ** beta /
                                         inv_distances[positions[ant]].sum() ** alpha + pheromones[positions[ant]].sum() ** beta)

            # Finding the next position with the highest probability
            next_position = np.argwhere(next_location_probability == np.amax(next_location_probability))[0][0]

            # Ensuring the next position has not already been visited by this ant
            while next_position in paths[:, ant]:
                # Setting the probability of the already visited node to zero
                next_location_probability[next_position] = 0.0

                # Finding the next highest probability node
                next_position = np.argwhere(next_location_probability == np.amax(next_location_probability))[0][0]

            # Adding the chosen next position to the ant's path
            paths[node, ant] = next_position

            # Updating the pheromone level on the path taken by the ant
            pheromones[node, next_position] += del_tau

    # Returning the paths taken by the ants, swapping axes to match the expected output format
    return np.swapaxes(paths, 0, 1)

<h1 style="color:brown;">6. Function to Run Ant Colony Optimization for TSP</h1>

In [10]:
def runAcoTsp(cities, iterations, colony, alpha, beta, del_tau, rho):
    # Calculating the inverse distance matrix for all city pairs
    inv_distances = inverseDistances(cities)
    
    # Raising the inverse distances to the power of beta to control the influence of distance in decision-making
    inv_distances = inv_distances ** beta

    # Initializing the pheromone matrix with zeros (no pheromones at the start)
    pheromones = np.zeros((cities.shape[0], cities.shape[0]))

    # Initializing variables to keep track of the minimum distance and corresponding path found
    min_distance = None
    min_path = None

    # Running the ACO algorithm for a specified number of iterations
    for i in range(iterations):
        # Randomly placing ants on the cities to initialize their starting positions
        positions = initializeAnts(cities, colony)
        
        # Moving the ants and finding paths based on pheromone levels and inverse distances
        paths = moveAnts(cities, positions, inv_distances, pheromones, alpha, beta, del_tau)

        # Applying pheromone evaporation across all paths
        pheromones *= (1 - rho)

        # Iterating over each path found by the ants in the current iteration
        for path in paths:
            distance = 0
            
            # Calculating the total distance of the path by summing the distances between consecutive cities
            for node in range(1, path.shape[0]):
                distance += np.sqrt(((cities[int(path[node])] - cities[int(path[node - 1])]) ** 2).sum())

            # Updating the minimum distance and corresponding path if a shorter path is found
            if not min_distance or distance < min_distance:
                min_distance = distance
                min_path = path

    # Appending the first city to the end of the minimum path to make it a closed loop (returning to the start)
    min_path = np.append(min_path, min_path[0])

    # Returning the minimum path and its distance as a tuple
    return (min_path, min_distance)

<h1 style="color:brown;">7. Retrieving TSP Data</h1>

In [6]:
# Retrieving TSP data from the file 'd200-14.tsp'
TSP = getTspData('d200-14.tsp')

# Converting the 'node_coord_section' of the TSP data into a NumPy array
cities = np.array(TSP['node_coord_section'])

<h1 style="color: brown;">8. Question 1: Effects of Heuristic Strength on the Quality of Solutions</h1>

<div style="background-color: #F5F5DC; border: 2px solid #8b4513; padding: 20px; border-radius: 10px; font-family: Arial, sans-serif; box-shadow: 3px 3px 15px rgba(0, 0, 0, 0.1);">

  
  <p>The heuristic strength in Ant Colony Optimization is influenced by two key parameters: Alpha and Beta. Alpha determines the weight of the pheromone trail in the decision making process. The primary goal of this experiment was to understand how varying these parameters affects the quality of the solutions produced by ACO. </p>

  <p> To examine the effects of heuristic strength, three values were selected for each parameter:</p>

  <ul style="padding-left: 20px;">
    <li><strong>Alpha :</strong> 0.1, 1, 10</li>
    <li><strong>Beta :</strong> 0.1, 1, 10</li>
  </ul>

  <p>The ACO algorithm was configured with the following parameters:</p>

  <ul style="padding-left: 20px;">
    <li><strong>Iterations:</strong> 50</li>
    <li><strong>Colony Size:</strong> 25 ants</li>
    <li><strong>Pheromone Deposit Amount :</strong> 1.5</li>
    <li><strong>Pheromone Evaporation Rate :</strong> 0.5</li>
  </ul>

  <p>For each combination of Alpha and Beta values, the ACO algorithm was executed 5 times to ensure reliable results. The average minimum distance across these 5 runs was calculated for each parameter combination and the standard deviation of these distances was also calculated to evaluate performance. The best combination was then selected based on the lowest average minimum distance obtained from these 5 runs.</p>

</div>


In [1]:
# Defining ACO parameters
iterations = 50  # Number of iterations for the ACO algorithm
colony = 25  # Number of ants in the colony
del_tau = 1.5  # Pheromone deposit amount 
rho = 0.5  # Pheromone evaporation rate 

# Defining ranges for alpha and beta values
alpha_values = [0.1, 1, 10]  
beta_values = [0.1, 1, 10] 

# Number of experiments to run for each parameter setting
num_runs = 5 

In [8]:
# Listing results for storing outcomes of different alpha and beta combinations
results = []

# Initializing variables for tracking the best alpha and beta values
best_alpha = None
best_beta = None
best_avg_distance = float('inf')  # Start with a large number for comparison

# Iterating through all combinations of alpha and beta values
for alpha in alpha_values:
    for beta in beta_values:
        # Printing the current combination of alpha and beta being tested
        print(f"\nRunning ACO with alpha={alpha} and beta={beta}")

        distances = []  # Storing the minimum distances for each run with the current alpha and beta
        
        # Performing multiple runs for each combination of alpha and beta
        for run in range(num_runs):
            # Running the ACO algorithm and getting the minimum distance of the current run
            _, min_distance = runAcoTsp(cities, iterations, colony, alpha, beta, del_tau, rho)
            distances.append(min_distance)  # Appending the result to the distances list
        
        # Calculating the average and standard deviation of distances across all runs
        average_distance = np.mean(distances)
        std_dev = np.std(distances)
        
        # Storing the results for the current combination of alpha and beta
        results.append((alpha, beta, average_distance, std_dev))

        # Updating the best combination if the current one has a better average distance
        if average_distance < best_avg_distance:
            best_alpha = alpha
            best_beta = beta
            best_avg_distance = average_distance


Running ACO with alpha=0.1 and beta=0.1

Running ACO with alpha=0.1 and beta=1

Running ACO with alpha=0.1 and beta=10

Running ACO with alpha=1 and beta=0.1

Running ACO with alpha=1 and beta=1

Running ACO with alpha=1 and beta=10

Running ACO with alpha=10 and beta=0.1

Running ACO with alpha=10 and beta=1

Running ACO with alpha=10 and beta=10


In [9]:
# Printing a summary of results for all combinations of alpha and beta
print("\nSummary of all results:")
for alpha, beta, avg_dist, std_dev in results:
    print(f"\nAlpha: {alpha}, Beta: {beta}")
    for run_num, distance in enumerate(distances, start=1):
        print(f"Run {run_num}: Minimum Distance = {distance:.2f}")
    print(f"Avg Minimum Distance of {num_runs} runs: {avg_dist:.2f}")
    print(f"Std Dev: {std_dev:.2f}")
    
# Printing the best combination of alpha and beta with the corresponding best average distance
print(f"\nBest Combination:")
print(f"Alpha: {best_alpha}, Beta: {best_beta} => Best Avg Minimum Distance of {num_runs} runs: {best_avg_distance:.2f}")


Summary of all results:

Alpha: 0.1, Beta: 0.1
Run 1: Minimum Distance = 20.12
Run 2: Minimum Distance = 19.57
Run 3: Minimum Distance = 19.95
Run 4: Minimum Distance = 20.12
Run 5: Minimum Distance = 20.12
Avg Minimum Distance of 5 runs: 19.98
Std Dev: 0.71

Alpha: 0.1, Beta: 1
Run 1: Minimum Distance = 20.12
Run 2: Minimum Distance = 19.57
Run 3: Minimum Distance = 19.95
Run 4: Minimum Distance = 20.12
Run 5: Minimum Distance = 20.12
Avg Minimum Distance of 5 runs: 19.59
Std Dev: 0.26

Alpha: 0.1, Beta: 10
Run 1: Minimum Distance = 20.12
Run 2: Minimum Distance = 19.57
Run 3: Minimum Distance = 19.95
Run 4: Minimum Distance = 20.12
Run 5: Minimum Distance = 20.12
Avg Minimum Distance of 5 runs: 20.44
Std Dev: 0.94

Alpha: 1, Beta: 0.1
Run 1: Minimum Distance = 20.12
Run 2: Minimum Distance = 19.57
Run 3: Minimum Distance = 19.95
Run 4: Minimum Distance = 20.12
Run 5: Minimum Distance = 20.12
Avg Minimum Distance of 5 runs: 16.98
Std Dev: 0.35

Alpha: 1, Beta: 1
Run 1: Minimum Distan

<h2 style="color:brown;">8.1 Results Summary</h2> 

<div style="background-color: #F5F5DC; border: 2px solid #8b4513; padding: 15px; border-radius: 10px; font-family: Arial, sans-serif; box-shadow: 3px 3px 15px rgba(0, 0, 0, 0.1);">
  
  <p style="font-size: 16px; color: #333;">The following table summarizes the results obtained from running ACO with various combinations of Alpha and Beta values:</p>



  <table style="width:100%; border-collapse: collapse; margin: 10px 0; font-size: 16px; color: #333;">
    <tr style="background-color: #8b4513; color: white;">
      <th style="padding: 8px; border: 1px solid #ddd;">Alpha </th>
      <th style="padding: 8px; border: 1px solid #ddd;">Beta </th>
      <th style="padding: 8px; border: 1px solid #ddd;">Run 1 Minimum Distance</th>
      <th style="padding: 8px; border: 1px solid #ddd;">Run 2 Minimum Distance</th>
      <th style="padding: 8px; border: 1px solid #ddd;">Run 3 Minimum Distance</th>
      <th style="padding: 8px; border: 1px solid #ddd;">Run 4 Minimum Distance</th>
      <th style="padding: 8px; border: 1px solid #ddd;">Run 5 Minimum Distance</th>
      <th style="padding: 8px; border: 1px solid #ddd;">Avg Minimum Distance</th>
      <th style="padding: 8px; border: 1px solid #ddd;">Std Dev</th>
    </tr>
    <tr>
      <td style="padding: 8px; border: 1px solid #ddd;">0.1</td>
      <td style="padding: 8px; border: 1px solid #ddd;">0.1</td>
      <td style="padding: 8px; border: 1px solid #ddd;">20.12</td>
      <td style="padding: 8px; border: 1px solid #ddd;">19.57</td>
      <td style="padding: 8px; border: 1px solid #ddd;">19.95</td>
      <td style="padding: 8px; border: 1px solid #ddd;">20.12</td>
      <td style="padding: 8px; border: 1px solid #ddd;">20.12</td>
      <td style="padding: 8px; border: 1px solid #ddd;">19.98</td>
      <td style="padding: 8px; border: 1px solid #ddd;">0.71</td>
    </tr>
    <tr>
      <td style="padding: 8px; border: 1px solid #ddd;">0.1</td>
      <td style="padding: 8px; border: 1px solid #ddd;">1</td>
      <td style="padding: 8px; border: 1px solid #ddd;">20.12</td>
      <td style="padding: 8px; border: 1px solid #ddd;">19.57</td>
      <td style="padding: 8px; border: 1px solid #ddd;">19.95</td>
      <td style="padding: 8px; border: 1px solid #ddd;">20.12</td>
      <td style="padding: 8px; border: 1px solid #ddd;">20.12</td>
      <td style="padding: 8px; border: 1px solid #ddd;">19.59</td>
      <td style="padding: 8px; border: 1px solid #ddd;">0.26</td>
    </tr>
    <tr>
      <td style="padding: 8px; border: 1px solid #ddd;">0.1</td>
      <td style="padding: 8px; border: 1px solid #ddd;">10</td>
      <td style="padding: 8px; border: 1px solid #ddd;">20.12</td>
      <td style="padding: 8px; border: 1px solid #ddd;">19.57</td>
      <td style="padding: 8px; border: 1px solid #ddd;">19.95</td>
      <td style="padding: 8px; border: 1px solid #ddd;">20.12</td>
      <td style="padding: 8px; border: 1px solid #ddd;">20.12</td>
      <td style="padding: 8px; border: 1px solid #ddd;">20.44</td>
      <td style="padding: 8px; border: 1px solid #ddd;">0.94</td>
    </tr>
    <tr>
      <td style="padding: 8px; border: 1px solid #ddd;">1</td>
      <td style="padding: 8px; border: 1px solid #ddd;">0.1</td>
      <td style="padding: 8px; border: 1px solid #ddd;">20.12</td>
      <td style="padding: 8px; border: 1px solid #ddd;">19.57</td>
      <td style="padding: 8px; border: 1px solid #ddd;">19.95</td>
      <td style="padding: 8px; border: 1px solid #ddd;">20.12</td>
      <td style="padding: 8px; border: 1px solid #ddd;">20.12</td>
      <td style="padding: 8px; border: 1px solid #ddd;">16.98</td>
      <td style="padding: 8px; border: 1px solid #ddd;">0.35</td>
    </tr>
    <tr>
      <td style="padding: 8px; border: 1px solid #ddd;">1</td>
      <td style="padding: 8px; border: 1px solid #ddd;">1</td>
      <td style="padding: 8px; border: 1px solid #ddd;">20.12</td>
      <td style="padding: 8px; border: 1px solid #ddd;">19.57</td>
      <td style="padding: 8px; border: 1px solid #ddd;">19.95</td>
      <td style="padding: 8px; border: 1px solid #ddd;">20.12</td>
      <td style="padding: 8px; border: 1px solid #ddd;">20.12</td>
      <td style="padding: 8px; border: 1px solid #ddd;">17.13</td>
      <td style="padding: 8px; border: 1px solid #ddd;">0.32</td>
    </tr>
    <tr>
      <td style="padding: 8px; border: 1px solid #ddd;">1</td>
      <td style="padding: 8px; border: 1px solid #ddd;">10</td>
      <td style="padding: 8px; border: 1px solid #ddd;">20.12</td>
      <td style="padding: 8px; border: 1px solid #ddd;">19.57</td>
      <td style="padding: 8px; border: 1px solid #ddd;">19.95</td>
      <td style="padding: 8px; border: 1px solid #ddd;">20.12</td>
      <td style="padding: 8px; border: 1px solid #ddd;">20.12</td>
      <td style="padding: 8px; border: 1px solid #ddd;">19.88</td>
      <td style="padding: 8px; border: 1px solid #ddd;">0.17</td>
    </tr>
    <tr>
      <td style="padding: 8px; border: 1px solid #ddd;">10</td>
      <td style="padding: 8px; border: 1px solid #ddd;">0.1</td>
      <td style="padding: 8px; border: 1px solid #ddd;">20.12</td>
      <td style="padding: 8px; border: 1px solid #ddd;">19.57</td>
      <td style="padding: 8px; border: 1px solid #ddd;">19.95</td>
      <td style="padding: 8px; border: 1px solid #ddd;">20.12</td>
      <td style="padding: 8px; border: 1px solid #ddd;">20.12</td>
      <td style="padding: 8px; border: 1px solid #ddd;">19.06</td>
      <td style="padding: 8px; border: 1px solid #ddd;">0.00</td>
    </tr>
    <tr>
      <td style="padding: 8px; border: 1px solid #ddd;">10</td>
      <td style="padding: 8px; border: 1px solid #ddd;">1</td>
      <td style="padding: 8px; border: 1px solid #ddd;">20.12</td>
      <td style="padding: 8px; border: 1px solid #ddd;">19.57</td>
      <td style="padding: 8px; border: 1px solid #ddd;">19.95</td>
      <td style="padding: 8px; border: 1px solid #ddd;">20.12</td>
      <td style="padding: 8px; border: 1px solid #ddd;">20.12</td>
      <td style="padding: 8px; border: 1px solid #ddd;">19.06</td>
      <td style="padding: 8px; border: 1px solid #ddd;">0.00</td>
    </tr>
    <tr>
      <td style="padding: 8px; border: 1px solid #ddd;">10</td>
      <td style="padding: 8px; border: 1px solid #ddd;">10</td>
      <td style="padding: 8px; border: 1px solid #ddd;">20.12</td>
      <td style="padding: 8px; border: 1px solid #ddd;">19.57</td>
      <td style="padding: 8px; border: 1px solid #ddd;">19.95</td>
      <td style="padding: 8px; border: 1px solid #ddd;">20.12</td>
      <td style="padding: 8px; border: 1px solid #ddd;">20.12</td>
      <td style="padding: 8px; border: 1px solid #ddd;">19.97</td>
      <td style="padding: 8px; border: 1px solid #ddd;">0.21</td>
    </tr>
  </table>

</div>


<h2 style="color:brown;">8.2 Observations and Analysis</h2> 

<div style="background-color: #F5F5DC; border: 2px solid #8b4513; padding: 15px; border-radius: 10px; font-family: Arial, sans-serif; box-shadow: 3px 3px 15px rgba(0, 0, 0, 0.1);">
  

  <p style="font-size: 16px; color: #333; text-align: justify;"><strong>Lower Alpha with Higher Beta:</strong></p>
  <p style="font-size: 16px; color: #333; text-align: justify;">Combinations such as Alpha = 0.1 and Beta = 10 resulted in the highest average minimum distances of 20.44 and the greatest standard deviations of 0.94. This suggests that an overemphasis on heuristic information, with low pheromone influence, lead to less stable results.</p>

  <p style="font-size: 16px; color: #333; text-align: justify;"><strong>Higher Alpha with Lower Beta:</strong></p>
  <p style="font-size: 16px; color: #333; text-align: justify;">For combinations like Alpha = 1 and Beta = 0.1, the average minimum distance was the lowest at 16.98, with a low standard deviation of 0.35. This indicates that a higher Alpha value, combined with lower Beta values, lead to more stable and optimal solutions.</p>

  <p style="font-size: 16px; color: #333; text-align: justify;"><strong>Balanced Alpha and Beta:</strong></p>
  <p style="font-size: 16px; color: #333; text-align: justify;">Intermediate values of Alpha and Beta (e.g., Alpha = 1 and Beta = 1) resulted in an average minimum distance of 17.13, with a standard deviation of 0.32. This combination provides a balance between pheromone influence and heuristic guidance, leading to a moderately good solution quality but not as effective as the best combinations observed.</p>

  <p style="font-size: 16px; color: #333; text-align: justify;"><strong>High Alpha with Higher Beta:</strong></p>
  <p style="font-size: 16px; color: #333; text-align: justify;">High Alpha values combined with high Beta values (e.g., Alpha = 10 and Beta = 10) resulted in an average minimum distance of 19.97 with a low standard deviation (0.21). While this combination yielded stable results, the solution quality was not as optimal as the best performing Alpha and Beta values.</p>

  <p style="font-size: 16px; color: #333; text-align: justify;"><strong>Conclusion:</strong></p>
  <p style="font-size: 16px; color: #333; text-align: justify;">In conclusion, the analysis shows that higher pheromone influence (Alpha) combined with lower heuristic guidance (Beta) leads to better and more stable solutions in ACO. The optimal parameter combination observed in this experiment was Alpha = 1 and Beta = 0.1, which provided the lowest average minimum distance and demonstrated the effectiveness of prioritizing pheromone information over heuristic guidance.</p>

</div>


<h2 style="color:brown;">8.3 Plausible Reasons for Observations</h2>  

<div style="background-color: #F5F5DC; border: 2px solid #8b4513; padding: 15px; border-radius: 10px; font-family: Arial, sans-serif; box-shadow: 3px 3px 15px rgba(0, 0, 0, 0.1);">
  
  <p style="font-size: 16px; color: #333; text-align: justify;"><strong>Exploration vs. Exploitation:</strong></p>
  <p style="font-size: 16px; color: #333; text-align: justify;">Higher Alpha values encourage ants to follow pheromone trails more closely, which helps in reinforcing good paths found. When combined with lower Beta values, this leads to better exploration and stability, as observed in the best results with Alpha = 1 and Beta = 0.1.</p>

  <p style="font-size: 16px; color: #333; text-align: justify;"><strong>Heuristic Information Over-Reliance:</strong></p>
  <p style="font-size: 16px; color: #333; text-align: justify;">High Beta values place more emphasis on the heuristic information (inverse distance). When combined with low Alpha values, it can lead to instability and higher average distances, as the search might become less focused on previously successful paths.</p>

  <p style="font-size: 16px; color: #333; text-align: justify;"><strong>Stability and Optimality:</strong></p>
  <p style="font-size: 16px; color: #333; text-align: justify;">Combinations with higher pheromone influence and lower heuristic weight generally produced more consistent results with lower average distances, suggesting that pheromone guidance plays a crucial role in finding optimal solutions in ACO.</p>

</div>


<h1 style="color:brown;">9. Question 2: Effects of Pheromone Evaporation Rate on the Quality of Solutions</h1>

<div style="background-color: #F5F5DC; border: 2px solid #8b4513; padding: 15px; border-radius: 10px; font-family: Arial, sans-serif; box-shadow: 3px 3px 15px rgba(0, 0, 0, 0.1);">
    
  <p style="font-size: 16px; color: #333; text-align: justify;"> Pheromone evaporation controls how quickly the pheromone trails decay over time. This experiment aimed to investigate how varying the pheromone evaporation rate affects the performance of the ACO algorithm. </p>

  <p style="font-size: 16px; color: #333; text-align: justify;"> To examine the effects of the pheromone evaporation rate, three values were selected:</p>
  
 <ul style="font-size: 16px; color: #333;">
    <li><strong>Rho:</strong> 0.1, 0.5, 0.9</li>
  </ul>
  <p style="font-size: 16px; color: #333; text-align: justify;">The ACO algorithm was configured with the following parameters:</p>

  <ul style="font-size: 16px; color: #333;">
    <li><strong>Iterations:</strong> 50</li>
    <li><strong>Colony Size:</strong> 25 ants</li>
    <li><strong>Alpha:</strong> 1 </li>
    <li><strong>Beta:</strong> 0.1</li>
    <li><strong>Pheromone Deposit Amount:</strong> 1.5</li>
  </ul>

  <p style="font-size: 16px; color: #333; text-align: justify;">For each value of Rho, the ACO algorithm was executed 5 times to ensure reliable results. The average minimum distance across these 5 runs was calculated and the standard deviation of these distances was also calculated to evaluate the impact of different pheromone evaporation rates. The best rate was then selected based on the lowest average minimum distance.</p>

</div>


In [12]:
# Defining ACO parameters
iterations = 50  # Number of iterations for the ACO algorithm
colony = 25  # Number of ants in the colony
alpha = 1  # Parameter controlling the influence of pheromones
beta = 0.1  # Parameter controlling the influence of distance
del_tau = 1.5  # Amount of pheromone deposited by each ant

# Defining different values for rho (pheromone evaporation rate) to test
rho_values = [0.1, 0.5, 0.9]
num_runs = 5  # Number of runs to perform for each value of rho

In [13]:
# Storing results for different rho values
results = []

# Initializing  variables to keep track of the best rho value and its corresponding average distance
best_rho = None
best_avg_distance = float('inf')  # Start with a large number to find the minimum

# Iterating through each value of rho
for rho in rho_values:
    # Printing the current rho value being tested
    print(f"\nRunning ACO with rho={rho}")
    
    distances = []  # Storing the minimum distances for each run with the current rho value
    
    # Performing multiple runs for the current rho value to get an average distance
    for run in range(num_runs):
        # Running the ACO algorithm and get the minimum distance of the current run
        _, min_distance = runAcoTsp(cities, iterations, colony, alpha, beta, del_tau, rho)
        distances.append(min_distance)  # Appending the result to the distances list
    
    # Calculating the average and standard deviation of the distances for the current rho value
    average_distance = np.mean(distances)
    std_dev = np.std(distances)
    
    # Storing the results for the current rho value
    results.append((rho, distances, average_distance, std_dev))

    # Updating the best rho value if the current one results in a lower average distance
    if average_distance < best_avg_distance:
        best_rho = rho
        best_avg_distance = average_distance


Running ACO with rho=0.1

Running ACO with rho=0.5

Running ACO with rho=0.9


In [14]:
# Printing a summary of results for all tested rho values
print("\nSummary of all results:")
for rho, distances, avg_dist, std_dev in results:
    print(f"\nRho: {rho}")
    for run_num, distance in enumerate(distances, start=1):
        print(f"Run {run_num}: Minimum Distance = {distance:.2f}")
    print(f"Avg Minimum Distance of {num_runs} runs: {avg_dist:.2f}")
    print(f"Std Dev: {std_dev:.2f}")

# Printing the best rho value and its corresponding best average distance
print(f"\nBest Rho Value:")
print(f"Rho: {best_rho} => Best Avg Minimum Distance of {num_runs} runs: {best_avg_distance:.2f}")


Summary of all results:

Rho: 0.1
Run 1: Minimum Distance = 17.19
Run 2: Minimum Distance = 17.44
Run 3: Minimum Distance = 17.90
Run 4: Minimum Distance = 16.57
Run 5: Minimum Distance = 18.06
Avg Minimum Distance of 5 runs: 17.43
Std Dev: 0.53

Rho: 0.5
Run 1: Minimum Distance = 17.44
Run 2: Minimum Distance = 17.37
Run 3: Minimum Distance = 17.22
Run 4: Minimum Distance = 17.10
Run 5: Minimum Distance = 17.08
Avg Minimum Distance of 5 runs: 17.24
Std Dev: 0.14

Rho: 0.9
Run 1: Minimum Distance = 16.22
Run 2: Minimum Distance = 16.86
Run 3: Minimum Distance = 16.58
Run 4: Minimum Distance = 17.26
Run 5: Minimum Distance = 17.38
Avg Minimum Distance of 5 runs: 16.86
Std Dev: 0.43

Best Rho Value:
Rho: 0.9 => Best Avg Minimum Distance of 5 runs: 16.86


<h2 style="color: brown;">9.1 Results Summary</h2>

<div style="background-color: #F5F5DC; border: 2px solid #8b4513; padding: 15px; border-radius: 10px; font-family: Arial, sans-serif; box-shadow: 3px 3px 15px rgba(0, 0, 0, 0.1);">
  
  <p style="font-size: 16px; color: #333;">The results of the experiment for different pheromone evaporation rates are summarized in the table below:</p>

  <table style="width:100%; border-collapse: collapse; margin: 10px 0; font-size: 16px; color: #333;">
    <tr style="background-color: #8b4513; color: white;">
      <th style="padding: 8px; border: 1px solid #ddd;">Rho </th>
      <th style="padding: 8px; border: 1px solid #ddd;">Run 1 Minimum Distance</th>
      <th style="padding: 8px; border: 1px solid #ddd;">Run 2 Minimum Distance</th>
      <th style="padding: 8px; border: 1px solid #ddd;">Run 3 Minimum Distance</th>
      <th style="padding: 8px; border: 1px solid #ddd;">Run 4 Minimum Distance</th>
      <th style="padding: 8px; border: 1px solid #ddd;">Run 5 Minimum Distance</th>
      <th style="padding: 8px; border: 1px solid #ddd;">Avg Minimum Distance</th>
      <th style="padding: 8px; border: 1px solid #ddd;">Std Dev</th>
    </tr>
    <tr>
      <td style="padding: 8px; border: 1px solid #ddd;">0.1</td>
      <td style="padding: 8px; border: 1px solid #ddd;">17.19</td>
      <td style="padding: 8px; border: 1px solid #ddd;">17.44</td>
      <td style="padding: 8px; border: 1px solid #ddd;">17.90</td>
      <td style="padding: 8px; border: 1px solid #ddd;">16.57</td>
      <td style="padding: 8px; border: 1px solid #ddd;">18.06</td>
      <td style="padding: 8px; border: 1px solid #ddd;">17.43</td>
      <td style="padding: 8px; border: 1px solid #ddd;">0.53</td>
    </tr>
    <tr>
      <td style="padding: 8px; border: 1px solid #ddd;">0.5</td>
      <td style="padding: 8px; border: 1px solid #ddd;">17.44</td>
      <td style="padding: 8px; border: 1px solid #ddd;">17.37</td>
      <td style="padding: 8px; border: 1px solid #ddd;">17.22</td>
      <td style="padding: 8px; border: 1px solid #ddd;">17.10</td>
      <td style="padding: 8px; border: 1px solid #ddd;">17.08</td>
      <td style="padding: 8px; border: 1px solid #ddd;">17.24</td>
      <td style="padding: 8px; border: 1px solid #ddd;">0.14</td>
    </tr>
    <tr>
      <td style="padding: 8px; border: 1px solid #ddd;">0.9</td>
      <td style="padding: 8px; border: 1px solid #ddd;">16.22</td>
      <td style="padding: 8px; border: 1px solid #ddd;">16.86</td>
      <td style="padding: 8px; border: 1px solid #ddd;">16.58</td>
      <td style="padding: 8px; border: 1px solid #ddd;">17.26</td>
      <td style="padding: 8px; border: 1px solid #ddd;">17.38</td>
      <td style="padding: 8px; border: 1px solid #ddd;">16.86</td>
      <td style="padding: 8px; border: 1px solid #ddd;">0.43</td>
    </tr>
  </table>

</div>


<h2 style="color:brown;">9.2 Observations and Analysis</h2>

<div style="background-color: #F5F5DC; border: 2px solid #8b4513; padding: 15px; border-radius: 10px; font-family: Arial, sans-serif; box-shadow: 3px 3px 15px rgba(0, 0, 0, 0.1);">
  
  <p style="font-size: 16px; color: #333; text-align: justify;"><strong>Low Pheromone Evaporation Rate (Rho = 0.1):</strong></p>
  <p style="font-size: 16px; color: #333; text-align: justify;">This setting resulted in the highest average minimum distance of 17.43 with the highest standard deviation of 0.53. The higher standard deviation indicates variability in the quality of solutions, which suggests that a low evaporation rate may cause pheromone trails to remain too strong for too long, leading to suboptimal convergence.</p>

  <p style="font-size: 16px; color: #333; text-align: justify;"><strong>Moderate Pheromone Evaporation Rate (Rho = 0.5):</strong></p>
  <p style="font-size: 16px; color: #333; text-align: justify;">An average minimum distance of 17.24 with the lowest standard deviation of 0.14 was observed with the moderate pheromone evaporation rate. This suggests that it provides a balance between maintaining useful pheromone information and preventing excessive trail reinforcement, leading to more consistent and reliable solutions.</p>

  <p style="font-size: 16px; color: #333; text-align: justify;"><strong>High Pheromone Evaporation Rate (Rho = 0.9):</strong></p>
  <p style="font-size: 16px; color: #333; text-align: justify;">This setting yielded the lowest average minimum distance of 16.86 with a standard deviation of 0.43. Although the standard deviation is slightly higher compared to Rho = 0.5, the average distance was the best. This indicates that a higher evaporation rate can be beneficial for achieving optimal solutions, as it prevents pheromone trails from dominating the search space for too long, promoting exploration.</p>

  <p style="font-size: 16px; color: #333; text-align: justify;"><strong>Conclusion:</strong></p>
  <p style="font-size: 16px; color: #333; text-align: justify;">The experiment demonstrates that the pheromone evaporation rate significantly affects the performance of the ACO algorithm. A higher evaporation rate (Rho = 0.9) was found to provide the best performance, achieving the lowest average minimum distance. This setting helps in maintaining a balance between exploration and exploitation, preventing the algorithm from being biased by outdated pheromone information.</p>

</div>


<h2 style="color:brown;">9.3 Plausible Reasons for Observations</h2>

<div style="background-color: #F5F5DC; border: 2px solid #8b4513; padding: 15px; border-radius: 10px; font-family: Arial, sans-serif; box-shadow: 3px 3px 15px rgba(0, 0, 0, 0.1);">
  
  <p style="font-size: 16px; color: #333; text-align: justify;"><strong>Low Evaporation Rate (Rho = 0.1):</strong></p>
  <p style="font-size: 16px; color: #333; text-align: justify;">When the evaporation rate is very low, pheromone trails persist for a long time. This can lead to premature convergence as ants may overly rely on outdated pheromone information, which could trap the algorithm in local optima.</p>

  <p style="font-size: 16px; color: #333; text-align: justify;"><strong>Moderate Evaporation Rate (Rho = 0.5):</strong></p>
  <p style="font-size: 16px; color: #333; text-align: justify;">A moderate evaporation rate allows pheromone information to decay at a balanced rate, helping the algorithm to maintain useful pheromone trails while still exploring new paths. This balance is crucial for achieving consistent and reliable solutions.</p>

  <p style="font-size: 16px; color: #333; text-align: justify;"><strong>High Evaporation Rate (Rho = 0.9):</strong></p>
  <p style="font-size: 16px; color: #333; text-align: justify;">A high evaporation rate ensures that pheromone trails do not last too long. This encourages continuous exploration of the solution space, reducing the risk of getting stuck in local optima and potentially leading to better overall solutions.</p>

</div>


<h1 style="color:brown;">10. Question 3: Comparison of Best Results: GA vs. ACO</h1>

<div style="background-color: #F5F5DC; border: 2px solid #8b4513; padding: 15px; border-radius: 10px; font-family: Arial, sans-serif; box-shadow: 3px 3px 15px rgba(0, 0, 0, 0.1);">
  
  <p style="font-size: 16px; color: #333; text-align: justify;">At last, GA and ACO were evaluated over ten runs to assess their effectiveness in minimizing the total distance traveled.</p>

  <p style="font-size: 16px; color: #333; text-align: justify;">For ACO, the best results were achieved with the following parameters:</p>
  <ul style="font-size: 16px; color: #333; text-align: justify; list-style-type: disc; padding-left: 20px;">
    <li><strong>Iterations:</strong> 50</li>
    <li><strong>Colony Size:</strong> 25</li>
    <li><strong>Alpha:</strong> 1 (Best value determined from previous experiments)</li>
    <li><strong>Beta:</strong> 0.1 (Best value determined from previous experiments)</li>
    <li><strong>Pheromone Deposit:</strong> 1.5</li>
    <li><strong>Rho:</strong> 0.9 (Best value determined from previous experiments)</li>
  </ul>

  <p style="font-size: 16px; color: #333; text-align: justify;">In Assignment 1, the GA was run, and the best results were achieved with the following parameters:</p>
  <ul style="font-size: 16px; color: #333; text-align: justify; list-style-type: disc; padding-left: 20px;">
   <li><strong>Population Size:</strong> 600 (three times the number of cities)</li>
    <li><strong>Maximum Generations:</strong> 500</li>
    <li><strong>Crossover Probability:</strong> 0.9</li>
    <li><strong>Mutation Probability:</strong> 0.2</li>
    <li><strong>Selection Method:</strong> Tournament Selection</li>
    <li><strong>Crossover Method:</strong> Ordered Crossover (OX)</li>
    <li><strong>Mutation Method:</strong> Shuffle Indexes Mutation</li>
  </ul>

</div>

In [7]:
# Retrieving TSP data from the file 'd200-14.tsp' 
TSP = getTspData('d200-14.tsp')

# Converting the 'node_coord_section' of the TSP data into a NumPy array
cities = np.array(TSP['node_coord_section'])

In [19]:
# Defining ACO parameters
iterations = 50  # Number of iterations for the ACO algorithm
colony = 25  # Number of ants in the colony
alpha = 1  # Parameter controlling the influence of pheromones 
beta = 0.1  # Parameter controlling the influence of distance 
del_tau = 1.5  # Amount of pheromone deposited by each ant on the path
rho = 0.9  # Pheromone evaporation rate 

# Initializing variables for tracking results
n = 10  # Number of times to run the ACO algorithm
distances = []  # List to store the minimum distances from each run
best_distance = float('inf')  # Initialize the best distance to a very large number
best_run = -1  # Variable to store the run number that produced the best distance
best_path = None  # Variable to store the path corresponding to the best distance

# Repeating the ACO process n times
for i in range(n):
    # Running the ACO algorithm and get the minimum path and distance
    min_path, min_distance = runAcoTsp(cities, iterations, colony, alpha, beta, del_tau, rho)
    distances.append(min_distance)  # Store the minimum distance of the current run
    
    # Updating the best distance and path if the current run's distance is better
    if min_distance < best_distance:
        best_distance = min_distance
        best_run = i + 1  # Run numbers are 1-based
        best_path = min_path

    # Printing the minimum distance found in the current run
    print(f"Minimum Distance of run {i + 1}: {min_distance:.2f}")

Minimum Distance of run 1: 16.02
Minimum Distance of run 2: 17.32
Minimum Distance of run 3: 16.70
Minimum Distance of run 4: 16.28
Minimum Distance of run 5: 17.47
Minimum Distance of run 6: 17.53
Minimum Distance of run 7: 17.32
Minimum Distance of run 8: 15.88
Minimum Distance of run 9: 17.77
Minimum Distance of run 10: 17.79


In [20]:
# Calculating the average distance across all runs
average_distance = np.mean(distances)

# Printing the average distance and the best distance found
print(f"\nAverage Distance of {n} runs: {average_distance:.2f}")
print(f"Best Distance found from run {best_run}: {best_distance:.2f}")


Average Distance of 10 runs: 17.01
Best Distance found from run 8: 15.88


<h2 style="color:brown;">10.1 Comment on Findings</h2>

<div style="background-color: #F5F5DC; border: 2px solid #8b4513; padding: 15px; border-radius: 10px; font-family: Arial, sans-serif; box-shadow: 3px 3px 15px rgba(0, 0, 0, 0.1);">

  <p style="font-size: 16px; color: #333;">The performance of ACO and GA is summarized in the following tables:</p>
    
  <h3 style="color: brown;">Table 1: ACO Results</h3>
    


  
  <table style="width:100%; border-collapse: collapse; margin: 10px 0; font-size: 16px; color: #333;">
    <tr style="background-color: #8b4513; color: white;">
      <th style="border: 1px solid #ddd; padding: 8px; text-align: left;">Run</th>
      <th style="border: 1px solid #ddd; padding: 8px; text-align: left;">Minimum Distance</th>
    </tr>
    <tr>
      <td style="border: 1px solid #ddd; padding: 8px;">1</td>
      <td style="border: 1px solid #ddd; padding: 8px;">16.02</td>
    </tr>
    <tr>
      <td style="border: 1px solid #ddd; padding: 8px;">2</td>
      <td style="border: 1px solid #ddd; padding: 8px;">17.32</td>
    </tr>
    <tr>
      <td style="border: 1px solid #ddd; padding: 8px;">3</td>
      <td style="border: 1px solid #ddd; padding: 8px;">16.70</td>
    </tr>
    <tr>
      <td style="border: 1px solid #ddd; padding: 8px;">4</td>
      <td style="border: 1px solid #ddd; padding: 8px;">16.28</td>
    </tr>
    <tr>
      <td style="border: 1px solid #ddd; padding: 8px;">5</td>
      <td style="border: 1px solid #ddd; padding: 8px;">17.47</td>
    </tr>
    <tr>
      <td style="border: 1px solid #ddd; padding: 8px;">6</td>
      <td style="border: 1px solid #ddd; padding: 8px;">17.53</td>
    </tr>
    <tr>
      <td style="border: 1px solid #ddd; padding: 8px;">7</td>
      <td style="border: 1px solid #ddd; padding: 8px;">17.32</td>
    </tr>
    <tr>
      <td style="border: 1px solid #ddd; padding: 8px;">8</td>
      <td style="border: 1px solid #ddd; padding: 8px;">15.88</td>
    </tr>
    <tr>
      <td style="border: 1px solid #ddd; padding: 8px;">9</td>
      <td style="border: 1px solid #ddd; padding: 8px;">17.77</td>
    </tr>
    <tr>
      <td style="border: 1px solid #ddd; padding: 8px;">10</td>
      <td style="border: 1px solid #ddd; padding: 8px;">17.79</td>
    </tr>
  </table>

  <p style="font-size: 16px; color: #333;">The best distance found using ACO was 15.88, achieved in Run 8.</p>

</div>


<div style="background-color: #F5F5DC; border: 2px solid #8b4513; padding: 15px; border-radius: 10px; font-family: Arial, sans-serif; box-shadow: 3px 3px 15px rgba(0, 0, 0, 0.1);">

  <h3 style="color: brown;">Table 2: GA Results</h3>

  <table style="width:100%; border-collapse: collapse; margin: 10px 0; font-size: 16px; color: #333;">
    <tr style="background-color: #8b4513; color: white;">
      <th style="border: 1px solid #ddd; padding: 8px; text-align: left;">Run</th>
      <th style="border: 1px solid #ddd; padding: 8px; text-align: left;">Best Fitness Distance</th>
    </tr>
    <tr>
      <td style="border: 1px solid #ddd; padding: 8px;">1</td>
      <td style="border: 1px solid #ddd; padding: 8px;">22.2546</td>
    </tr>
    <tr>
      <td style="border: 1px solid #ddd; padding: 8px;">2</td>
      <td style="border: 1px solid #ddd; padding: 8px;">22.2533</td>
    </tr>
    <tr>
      <td style="border: 1px solid #ddd; padding: 8px;">3</td>
      <td style="border: 1px solid #ddd; padding: 8px;">22.2533</td>
    </tr>
    <tr>
      <td style="border: 1px solid #ddd; padding: 8px;">4</td>
      <td style="border: 1px solid #ddd; padding: 8px;">22.2533</td>
    </tr>
    <tr>
      <td style="border: 1px solid #ddd; padding: 8px;">5</td>
      <td style="border: 1px solid #ddd; padding: 8px;">21.6109</td>
    </tr>
    <tr>
      <td style="border: 1px solid #ddd; padding: 8px;">6</td>
      <td style="border: 1px solid #ddd; padding: 8px;">20.3838</td>
    </tr>
    <tr>
      <td style="border: 1px solid #ddd; padding: 8px;">7</td>
      <td style="border: 1px solid #ddd; padding: 8px;">20.3838</td>
    </tr>
    <tr>
      <td style="border: 1px solid #ddd; padding: 8px;">8</td>
      <td style="border: 1px solid #ddd; padding: 8px;">20.3838</td>
    </tr>
    <tr>
      <td style="border: 1px solid #ddd; padding: 8px;">9</td>
      <td style="border: 1px solid #ddd; padding: 8px;">20.3838</td>
    </tr>
    <tr>
      <td style="border: 1px solid #ddd; padding: 8px;">10</td>
      <td style="border: 1px solid #ddd; padding: 8px;">20.3838</td>
    </tr>
  </table>

  <p style=" font-size: 16px; color: #333;">The best distance found using GA was 20.3838, consistently observed in Runs 6 through 10.</p>

</div>


<div style="background-color: #F5F5DC; border: 2px solid #8b4513; padding: 15px; border-radius: 10px; font-family: Arial, sans-serif; box-shadow: 3px 3px 15px rgba(0, 0, 0, 0.1);">
  
  <p style="font-size: 16px; color: #333; text-align: justify;">
The comparison reveals that ACO significantly outperformed GA in this TSP instance. The best distance found by ACO was 15.88, which was approximately 5 units shorter than the best distance found by GA at 20.3838. This difference indicates that ACO was more effective in finding a shorter and therefore more optimal route among the 200 cities.</p>

  <p style="font-size: 16px; color: #333; text-align: justify;">It is evident that GA displayed a high degree of consistency from Runs 6 to 10, with the same distance achieved in these runs. However, this stability indicates that GA may have reached a local optimum and did not find a better solution within the provided runs. On the other hand, ACO's variability in results, while showing a broader range, enabled it to achieve a better overall solution. ACO's ability to explore the solution space more effectively allowed it to avoid local optima and find a globally better solution.</p>



</div>


<h1 style="color:brown;">11 Conclusion</h1>

<div style="background-color: #F5F5DC; border: 2px solid #8b4513; padding: 15px; border-radius: 10px; font-family: Arial, sans-serif; box-shadow: 3px 3px 15px rgba(0, 0, 0, 0.1);">
  
  <p style="font-size: 16px; color: #333; text-align: justify;">
The experiments conducted highlight the effectiveness of ACO over GA in solving the Traveling Salesman Problem for this instance. In Experiment 1, it was observed that a higher pheromone influence (Alpha = 1) combined with lower heuristic guidance (Beta = 0.1) produced the most stable and optimal solutions, achieving the lowest average minimum distance. This highlights the importance of prioritizing pheromone information over heuristic data for better solution quality in ACO.</p>

  <p style="font-size: 16px; color: #333; text-align: justify;"> In Experiment 2, it was observed that the pheromone evaporation rate plays a critical role in the performance of ACO. A higher evaporation rate (Rho = 0.9) yielded the best results, as it helped maintain a balance between exploration and exploitation by preventing the algorithm from relying too heavily on outdated pheromone trails.</p>

  <p style="font-size: 16px; color: #333; text-align: justify;">The comparison between ACO and GA further emphasized ACO's superiority in this TSP instance. ACO found a best distance of 15.88, which was approximately 5 units shorter than GA's best distance of 20.3838. </p>
      
  <p style="font-size: 16px; color: #333; text-align: justify;">These results indicate that ACO is a more effective and suitable algorithm for complex optimization problems like the TSP. </p>

</div>
